In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

machine_id_inicio = 21
machine_id_fim = 30
output_final_csv = 'melhores_modelos_finais_arima_sarima.csv'
output_todos_csv = 'todos_modelos_avaliados_arima_sarima.csv'

print("Carregando dados...")
dataset_url = 'https://raw.githubusercontent.com/marcelowf/LineNex.Dataset/main/dataset_preprocessado.csv'
df_full = pd.read_csv(dataset_url, parse_dates=['Timestamp'])

best_param_url = 'https://raw.githubusercontent.com/marcelowf/LineNex.Dataset/main/melhores_modelos_arima_por_maquina.csv'
melhores_arima = pd.read_csv(best_param_url)

melhores_arima = melhores_arima[(melhores_arima['Machine_ID'] >= machine_id_inicio) &
                                 (melhores_arima['Machine_ID'] <= machine_id_fim)]

seasonal_orders = [(1, 1, 1, s) for s in [24, 12, 7]]
resultados_finais, todos_resultados = [], []

for idx, row in melhores_arima.iterrows():
    machine_id = row['Machine_ID']
    variable = row['Variable']
    arima_order = (int(row['p']), int(row['d']), int(row['q']))
    df_machine = df_full[df_full['Machine_ID'] == machine_id].copy()

    series = pd.to_numeric(df_machine[variable], errors='coerce').dropna().reset_index(drop=True)

    if len(series) < 30:
        print(f"[{idx}] Ignorado - Poucos dados para Machine {machine_id}, Variável {variable}")
        continue

    print(f"[{idx}] Avaliando Machine {machine_id}, Variável '{variable}' - ARIMA{arima_order}")

    # ARIMA
    arima_aic = arima_bic = arima_mse = np.inf
    model_arima = None
    try:
        model_arima = ARIMA(series, order=arima_order).fit()
    except Exception as e:
        print(f"Primeira tentativa ARIMA falhou: {e}")
        try:
            model_arima = ARIMA(series, order=arima_order).fit(method="yule_walker", start_params=None)
        except Exception as e2:
            print(f"Segunda tentativa ARIMA falhou: {e2}")
            model_arima = None

    if model_arima:
        arima_aic = model_arima.aic
        arima_bic = model_arima.bic
        try:
            fitted = model_arima.fittedvalues
            min_len = min(len(series), len(fitted))
            arima_mse = mean_squared_error(series[-min_len:], fitted[-min_len:])
        except Exception:
            arima_mse = np.nan

    todos_resultados.append({
        'Machine_ID': machine_id,
        'Variable': variable,
        'Model': 'ARIMA',
        'ARIMA_Order': arima_order,
        'Seasonal_Order': None,
        'AIC': arima_aic,
        'BIC': arima_bic,
        'MSE': arima_mse,
        'n_obs': len(series)
    })

    best_model_type = 'ARIMA'
    best_order = arima_order
    best_seasonal_order = None
    best_aic, best_bic, best_mse = arima_aic, arima_bic, arima_mse

    # SARIMA
    for seasonal_order in seasonal_orders:
        try:
            model_sarima = SARIMAX(series, order=arima_order, seasonal_order=seasonal_order,
                                   enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
            fitted = model_sarima.fittedvalues
            min_len = min(len(series), len(fitted))
            mse = mean_squared_error(series[-min_len:], fitted[-min_len:])

            todos_resultados.append({
                'Machine_ID': machine_id,
                'Variable': variable,
                'Model': 'SARIMA',
                'ARIMA_Order': arima_order,
                'Seasonal_Order': seasonal_order,
                'AIC': model_sarima.aic,
                'BIC': model_sarima.bic,
                'MSE': mse,
                'n_obs': len(series)
            })

            if model_sarima.aic < best_aic:
                best_model_type = 'SARIMA'
                best_seasonal_order = seasonal_order
                best_aic = model_sarima.aic
                best_bic = model_sarima.bic
                best_mse = mse
        except Exception as e:
            print(f"SARIMA {seasonal_order} falhou: {e}")
            continue

    print(f"Melhor modelo para Machine {machine_id}, Variável '{variable}': {best_model_type} {best_order} {'+ ' + str(best_seasonal_order) if best_seasonal_order else ''} | AIC: {best_aic:.2f}, MSE: {best_mse:.4f}")

    resultados_finais.append({
        'Machine_ID': machine_id,
        'Variable': variable,
        'Best_Model': best_model_type,
        'ARIMA_Order': best_order,
        'Seasonal_Order': best_seasonal_order,
        'AIC': best_aic,
        'BIC': best_bic,
        'MSE': best_mse,
        'n_obs': len(series)
    })

    pd.DataFrame(resultados_finais).replace([np.inf, -np.inf], np.nan).to_csv(output_final_csv, index=False)
    pd.DataFrame(todos_resultados).replace([np.inf, -np.inf], np.nan).to_csv(output_todos_csv, index=False)
    print(f"Dados salvos após Machine {machine_id}")

print("\nProcesso concluído")

Carregando dados...
[5] Avaliando Machine 29, Variável 'Production_Speed_units_per_hr' - ARIMA(0, 1, 2)
→ Melhor modelo para Machine 29, Variável 'Production_Speed_units_per_hr': SARIMA (0, 1, 2) + (1, 1, 1, 24) | AIC: 25071.54, MSE: 17586.9149
✓ Dados salvos após Machine 29
[6] Avaliando Machine 29, Variável 'Predictive_Maintenance_Score' - ARIMA(0, 0, 1)
→ Melhor modelo para Machine 29, Variável 'Predictive_Maintenance_Score': ARIMA (0, 0, 1)  | AIC: 743.44, MSE: 0.0840
✓ Dados salvos após Machine 29
[7] Avaliando Machine 29, Variável 'Efficiency_Status' - ARIMA(0, 0, 0)
→ Melhor modelo para Machine 29, Variável 'Efficiency_Status': ARIMA (0, 0, 0)  | AIC: 2422.72, MSE: 0.1912
✓ Dados salvos após Machine 29
[8] Avaliando Machine 29, Variável 'Quality_Control_Defect_Rate_%' - ARIMA(3, 0, 2)
→ Melhor modelo para Machine 29, Variável 'Quality_Control_Defect_Rate_%': SARIMA (3, 0, 2) + (1, 1, 1, 24) | AIC: 9994.82, MSE: 8.9458
✓ Dados salvos após Machine 29
[9] Avaliando Machine 29, Vari